In [12]:
import pandas as pd
import numpy as np

In [13]:
ratings = pd.read_csv('interactions.csv')
paths=pd.read_csv('paths.csv')
users=pd.read_csv('users.csv')

In [14]:
paths.head()

,time_zone,begin,end,pathId,title
0,28:14.0,1.460000e+12,1.460000e+12,1,path1
1,06:44.0,1.460000e+12,1.460000e+12,2,path2
2,41:57.0,1.460000e+12,1.460000e+12,3,path3
3,48:30.0,1.460000e+12,1.460000e+12,4,path4
4,07:58.0,1.460000e+12,1.460000e+12,5,path5


In [15]:
ratings.head()

,id,user,path,rating
0,491,1,3,3.741418
1,3326,1,5,1.323232
2,1164,1,13,3.057498
3,4132,1,13,4.008407
4,3159,1,18,3.901063


In [16]:
ratings= ratings.drop_duplicates(subset=['user', 'path'])
R_df = ratings.pivot(index = 'user', columns ='path', values = 'rating').fillna(0)
R_df.head()

path,1,2,3,4,5,6,7,8,9,10,...,427,428,429,430,431,432,433,434,435,436
user,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.0,3.741418,0.000000,1.323232,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.462907,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,1.173099,0.0,0.000000,0.000000,0.000000,2.291721,0.000000,3.355627,0.000000,0.000000,...,0.000000,0.2778,0.0,0.476646,4.238373,0.000000,0.000000,0.0,0.000000,1.830069
3,0.000000,0.0,0.000000,0.000000,0.000000,0.263936,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.0,0.000000,0.000000,2.557813,1.425198,0.0,0.000000,0.000000
4,0.000000,0.0,1.482334,4.899022,0.000000,0.000000,3.104816,0.000000,4.607402,3.088321,...,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,4.376799,0.000000
5,0.000000,0.0,0.000000,0.000000,1.805102,2.400654,0.000000,1.863695,0.000000,0.130695,...,2.158030,0.0000,0.0,0.000000,2.302673,0.000000,0.000000,0.0,0.000000,0.000000


In [17]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

C:\Users\marco\Anaconda3\envs\GIS\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [85]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 49)

In [86]:
sigma=np.diag(sigma)

In [87]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [88]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
preds_df.head()

path,1,2,3,4,5,6,7,8,9,10,...,427,428,429,430,431,432,433,434,435,436
0,0.021710,-0.054073,3.699179,-0.002658,1.300461,-0.005741,0.016092,-0.022421,-0.007990,-0.041321,...,2.477345,0.019538,-0.039294,-0.002733,-0.022109,-0.002574,0.020766,-0.034706,0.017336,-0.035460
1,1.254138,-0.201843,-0.157671,-0.009922,-0.084997,2.270290,0.060066,3.271935,-0.029824,-0.154242,...,0.053897,0.350732,-0.146677,0.466445,4.155843,-0.009607,0.077513,-0.129550,0.064711,1.697706
2,-0.143698,0.357907,0.279581,0.017594,0.150716,0.301937,-0.106509,0.148402,0.052883,0.273501,...,-0.095570,-0.129323,0.260087,0.018088,0.146341,2.574848,1.287752,0.229717,-0.114745,0.234706
3,-0.098624,0.245642,1.674218,4.911098,0.103441,0.026081,3.031716,0.101853,4.643697,3.276033,...,-0.065592,-0.088758,0.178505,0.012414,0.100438,0.011691,-0.094333,0.157661,4.298047,0.161085
4,0.039507,-0.098400,-0.076866,-0.004837,1.763665,2.390207,0.029283,1.822894,-0.014539,0.055501,...,2.184306,0.035555,-0.071506,-0.004973,2.262439,-0.004683,0.037788,-0.063156,0.031547,-0.064528


In [89]:
def recommend_movies(predictions_df, user, paths_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = user - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.user == (user)]
    user_full = (user_data.merge(paths_df, how = 'left', left_on = 'path', right_on = 'pathId').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(user, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (paths_df[~paths_df['pathId'].isin(user_full['path'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'pathId',
               right_on = 'path').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [90]:
already_rated, predictions = recommend_movies(preds_df, 40, paths, ratings, 10)

User 40 has already rated 105 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [91]:
already_rated.head(10)

,id,user,path,rating,time_zone,begin,end,pathId,title
8,3986,40,31,4.786470,48:37.0,1.470000e+12,1.470000e+12,31,path31
5,2377,40,23,4.727726,36:53.0,1.470000e+12,1.470000e+12,23,path23
18,1918,40,72,4.661592,35:40.0,1.490000e+12,1.490000e+12,72,path72
95,1428,40,417,4.658785,33:53.0,1.510000e+12,1.510000e+12,417,path417
104,4295,40,436,4.645592,44:16.0,1.510000e+12,1.510000e+12,436,path436
96,229,40,420,4.560478,00:04.0,1.510000e+12,1.510000e+12,420,path420
72,2320,40,292,4.466834,35:32.0,1.490000e+12,1.490000e+12,292,path292
48,2723,40,174,4.434261,45:36.0,1.490000e+12,1.490000e+12,174,path174
103,3255,40,436,4.354368,44:16.0,1.510000e+12,1.510000e+12,436,path436
62,322,40,241,4.287274,19:35.0,1.490000e+12,1.490000e+12,241,path241


In [92]:
predictions

,time_zone,begin,end,pathId,title,path
41,20:48.0,1.480000e+12,1.480000e+12,54,path54,54.0
258,34:22.0,1.490000e+12,1.490000e+12,328,path328,328.0
316,58:48.0,1.500000e+12,1.500000e+12,402,path402,402.0
138,09:19.0,1.490000e+12,1.490000e+12,183,path183,183.0
0,06:44.0,1.460000e+12,1.460000e+12,2,path2,2.0
335,21:21.0,1.510000e+12,1.510000e+12,426,path426,426.0
274,41:32.0,1.500000e+12,1.500000e+12,350,path350,350.0
142,20:24.0,1.490000e+12,1.490000e+12,187,path187,187.0
266,11:35.0,1.500000e+12,1.500000e+12,336,path336,336.0
293,31:49.0,1.500000e+12,1.500000e+12,372,path372,372.0
